In [0]:
#We are executing the script in databricks notebook

In [0]:
#reading the dataframe 
xml_df=spark.read.text("/FileStore/tables/file-1.xml")

In [0]:
#when we see the we will get the single column value
xml_df.show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
value |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
<?xml version="1.0" encoding="UTF-8"?> |
<rss version="2.0"> |
 <channel> |
 <title>Top stories - European Parliament</title> |
 <link>https://www.europarl.europa.eu/news/en</link> |
 <description /> |
 <language>EN</language> |
 <copyright>� European Union, 2021 - EP</copyright> |
 <pubDate>Tue, 21 Sep 2021 13:07:46 GMT</pubDate> |
 <item> |
 <title>Top story - State of the European Union Debate 2021 - Addressing Europeans' concerns</title> |
 <link>https://www.europarl.europa.eu/news/en/headlines/priorities/soteu2021</link> |
 <description>&lt;img src="https://www.europarl.europa.eu/resources/library/images/20210913PHT12318/20210913PHT12318-ms.jpg" align="left" width="125" hspace="5" vspace="0" /&gt;The most important EU debate of the year, looking at last year�s achievements as well as plans and vision for the future, took place on 15 September.&lt;br /&gt; &lt;br /&gt;Source : &lt;a href="https://www.europarl.europa.eu/privacy-policy/en"&gt;� European Union, 2021 - EP&lt;/a&gt;</description>|
 <source url="https://www.europarl.europa.eu/rss/doc/top-stories/en.xml">Top stories - European Parliament</source> |
 <category domain="type">Top story</category> |
 <pubDate>Mon, 13 Sep 2021 13:03:26 GMT</pubDate> |
 <guid isPermaLink="false">TST_TST-2021-09-09-11812_EN</guid> |
 </item> |
 <item> |
 <title>Top story - State of the European Union Debate 2021 - Addressing Europeans' concerns</title> |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 20 rows

In [0]:
#using the spark.xml package we are using row tag and root tag to read item each record
xmldf_with_schema=spark.read.format("xml").option("rootTag","channel").option("rowTag","item").option("inferSchema",True).load("/FileStore/tables/file-1.xml")

In [0]:
#we need to flateen the struct type since we can't write struct type to csv 

In [0]:
def flatten_struct(schema, prefix=""):
    result = []
    for elem in schema:
        if isinstance(elem.dataType, StructType):
            result += flatten_struct(elem.dataType, prefix + elem.name + ".")
        else:
            result.append(col(prefix + elem.name).alias(prefix + elem.name))
    return result


In [0]:
cols=xmldf_with_schema.columns

In [0]:
#create the flattend dataframe
from pyspark.sql.functions import col
from pyspark.sql.types import StructType
df_expanded = xmldf_with_schema.select(*cols)
df_flattened = df_expanded.select(flatten_struct(df_expanded.schema))

In [0]:
df_flattened.show()

+---------------+----------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
category._VALUE|category._domain| description| guid._VALUE|guid._isPermaLink| link| pubDate| source._VALUE| source._url| title|
+---------------+----------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 Top story| type|<img src="https:/...|TST_TST-2021-09-0...| false|https://www.europ...|Mon, 13 Sep 2021 ...|Top stories - Eur...|https://www.europ...|Top story - State...|
 story| type|<img src="https:/...|TST_TST-2021-09-0...| false|https://www.europ...|Mon, 13 Sep 2021 ...|Top stories - Eur...|https://www.europ...|Top story - State...|
 Top story| type|<img src="https:/...|TST_TST-2021-09-0...| false|https://www.europ...|Mon, 13 Sep 2021 ...|Top stories - Eur...|https://www.europ...|Top story - State...|
+---------------+----------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
#fliter only the Top Story
df_top_story_filtered=df_flattened.filter(col("Category._VALUE")=="Top story")

In [0]:
display(df_top_story_filtered)

category._VALUE,category._domain,description,guid._VALUE,guid._isPermaLink,link,pubDate,source._VALUE,source._url,title
Top story,type,"The most important EU debate of the year, looking at last year�s achievements as well as plans and vision for the future, took place on 15 September. Source : � European Union, 2021 - EP",TST_TST-2021-09-09-11812_EN,false,https://www.europarl.europa.eu/news/en/headlines/priorities/soteu2021,"Mon, 13 Sep 2021 13:03:26 GMT",Top stories - European Parliament,https://www.europarl.europa.eu/rss/doc/top-stories/en.xml,Top story - State of the European Union Debate 2021 - Addressing Europeans' concerns
Top story,type,"The most important EU debate of the year, looking at last year�s achievements as well as plans and vision for the future, took place on 15 September. Source : � European Union, 2021 - EP",TST_TST-2021-09-09-11812_EN,false,https://www.europarl.europa.eu/news/en/headlines/priorities/soteu2021,"Mon, 13 Sep 2021 13:03:26 GMT",Top stories - European Parliament,https://www.europarl.europa.eu/rss/doc/top-stories/en.xml,Top story - State of the European Union Debate 2021 - Addressing Europeans' concerns


In [0]:
#this is the path to save
path_to_save="dbfs:/xml_items_top_stories_news.csv"

In [0]:
df_top_story_filtered.coalesce(1).write.option("inferSchema","true").option("header","true").format("csv").save(path_to_save)

In [0]:
list_of_files=dbutils.fs.ls(path_to_save)

In [0]:
list_of_files

[FileInfo(path='dbfs:/xml_items_top_stories_news.csv/_SUCCESS', name='_SUCCESS', size=0),
 FileInfo(path='dbfs:/xml_items_top_stories_news.csv/_committed_4438668309106954532', name='_committed_4438668309106954532', size=112),
 FileInfo(path='dbfs:/xml_items_top_stories_news.csv/_started_4438668309106954532', name='_started_4438668309106954532', size=0),
 FileInfo(path='dbfs:/xml_items_top_stories_news.csv/part-00000-tid-4438668309106954532-2926a155-81a5-4ced-8349-2d0245eb5084-40-1-c000.csv', name='part-00000-tid-4438668309106954532-2926a155-81a5-4ced-8349-2d0245eb5084-40-1-c000.csv', size=1650)]

In [0]:
#create only one csv file named export.csv to save all the top story rows
file = dbutils.fs.ls(path_to_save)[-1].path
dbutils.fs.cp(file, "dbfs:/export.csv")
dbutils.fs.rm(path_to_save, recurse=True)

True

In [0]:
dbutils.fs.ls("export.csv")

[FileInfo(path='dbfs:/export.csv', name='export.csv', size=1650)]

In [0]:
#after storing the export.csv we need to read the export.csv is everything is fine
df=spark.read.option("header",True).csv("dbfs:/export.csv")

In [0]:
display(df)

category._VALUE,category._domain,description,guid._VALUE,guid._isPermaLink,link,pubDate,source._VALUE,source._url,title
Top story,type,"The most important EU debate of the year, looking at last year�s achievements as well as plans and vision for the future, took place on 15 September. Source : � European Union, 2021 - EP",TST_TST-2021-09-09-11812_EN,false,https://www.europarl.europa.eu/news/en/headlines/priorities/soteu2021,"Mon, 13 Sep 2021 13:03:26 GMT",Top stories - European Parliament,https://www.europarl.europa.eu/rss/doc/top-stories/en.xml,Top story - State of the European Union Debate 2021 - Addressing Europeans' concerns
Top story,type,"The most important EU debate of the year, looking at last year�s achievements as well as plans and vision for the future, took place on 15 September. Source : � European Union, 2021 - EP",TST_TST-2021-09-09-11812_EN,false,https://www.europarl.europa.eu/news/en/headlines/priorities/soteu2021,"Mon, 13 Sep 2021 13:03:26 GMT",Top stories - European Parliament,https://www.europarl.europa.eu/rss/doc/top-stories/en.xml,Top story - State of the European Union Debate 2021 - Addressing Europeans' concerns
